ANALISIS DE CONTINGENCIAS CON PYTHON

Codigo elaborado por: Washington Soto
Para el concuros de Papers CONEIMERA 2023
e-mail: washi.17.01@gmail.com

In [37]:
#Librerias utilizadas
import numpy as np
import pandas as pd

Funcion para determinar la matriz B 

In [38]:
def calculate_B_matrix(linedata, Sbase, Vbase):
    j = 1j
    nbr = int(len(linedata[:, 0]))
    nbus = int(max(max(linedata[:, 0]
                       ), max(linedata[:, 1])))
    B_matrix = np.zeros((nbus, nbus), dtype=np.float64) 

    # Paso 1: Calcular Zbase, pero al tener los datos en pu, Zbase = 1
    Zbase = 1

    # Pase 2: Armar la matriz B
    B_values = Zbase / linedata[:, 3]

    for k in range(nbr):
        nl = int(linedata[k, 0])
        nr = int(linedata[k, 1])

        # Paso 3: Calcular los elementos de la matriz B
        B_matrix[nl - 1, nl - 1] += B_values[k]
        B_matrix[nr - 1, nr - 1] += B_values[k]
        B_matrix[nl - 1, nr - 1] -= B_values[k]
        B_matrix[nr - 1, nl - 1] -= B_values[k]
    # Remueve la fila y columna de la barra slack
    B_matrix_result = B_matrix[1:, 1:]
    return B_matrix, B_matrix_result

Funcion para determinar el flujo DC inicial

In [39]:
#Calculo de theta
def calculate_theta(B_matrix_result, potencia_barras):
    Theta = np.linalg.inv(B_matrix_result) @ potencia_barras[:,1:] / Sbase
    Theta = np.vstack((np.array([[0]]),Theta))
    return Theta
#Calculo del flujo de potencia DC inicial
def calculate_dc_power_flows(B_matrix, linedata):
    dc_power_flows = []
    for k in range(len(linedata)):
        nl = int(linedata[k, 0])  # De-bus
        nr = int(linedata[k, 1])  # A-bus
        
        # Obtenemos el valor de theta en los buses De-bus y A-bus
        theta_from = Theta[nl-1]
        theta_to = Theta[nr-1]
        # Se calcula el flujo DC para el elemento con la formula P = B * (theta_from - theta_to)
        B = B_matrix[nl - 1, nr - 1]
        P = -B * (theta_from - theta_to)
        dc_power_flows.append(P)

    dc_power_flows = np.array(dc_power_flows)
    return dc_power_flows

Funcion para determinar los coeficientes ali (Contingencias ante salida de generador)

In [40]:
#Calculo de F
def calculate_F(B_matrix_result):
    F = modified_matrix = np.pad(np.linalg.inv(B_matrix_result), ((1, 0), (1, 0)), mode='constant')
    return F
#Calculo de coeficientes ali
def calculate_ali(linedata,B_matrix,F,ni):
    ali = []
    for k in range(len(linedata)):
        nl = int(linedata[k, 0])  # De-bus
        nr = int(linedata[k, 1])  # A-bus

        ali.append(1/(1/-B_matrix[nl-1,nr-1])*(F[nl-1,ni-1]-F[nr-1,ni-1]))
    ali = np.array(ali)[:, np.newaxis]
    return ali
#Nuevo flujo de potencia tras la salida del generador, por factores de sensibilidad
def calculate_new_dc_power_flows(linedata, dc_power_flows, potencia_barras,Sbase,ni,ali):
    new_dc_power_flows = []
    for k in range(len(linedata)):
        nl = int(linedata[k, 0])  
        nr = int(linedata[k, 1])  
        NP = dc_power_flows[k] + ali[k] * -potencia_barras[ni-2,1]/Sbase 
        new_dc_power_flows.append(NP)
    new_dc_power_flows = np.array(new_dc_power_flows)
    return new_dc_power_flows

Funcion para determinar los coeficientes dlk (Contingencias ante salida de generador)

In [41]:
#Calculo de factores de sencibilidad dlk
def calculate_dlk(linedata, B_matrix, F):
    dlk = []
    for k in range(len(linedata)):
        ni = int(linedata[k, 0])  # De-bus
        nj = int(linedata[k, 1])  # A-bus
        nm = 0 #Salida de linea 1 [1-2]
        nn = 1 #Salida de linea 2
        dlk.append(1/(1/(-B_matrix[ni-1,nj-1]))*((1/(-B_matrix[nm,nn])*(F[ni-1,nm]-F[nj-1,nm]-F[ni-1,nn]+F[nj-1,nn]))/(1/(-B_matrix[nm,nn])-(F[nm,nm]+F[nn,nn]-2*F[nm,nn]))))
    dlk = np.array(dlk)[:, np.newaxis]
    return dlk
#Nuevo flujo de potencia tras la salida de la linea, por factores de sensibilidad
def new_dc_power_flows_linea(linedata, dc_power_flows, dlk, nm, nn):
    new_dc_power_flows = []
    for k in range(len(linedata)):
        nl = int(linedata[k, 0])  # From-bus
        nr = int(linedata[k, 1])  # To-bus
        NP = dc_power_flows[k] + dlk[k] * dc_power_flows[1] 
        new_dc_power_flows.append(NP)
    new_dc_power_flows = np.array(new_dc_power_flows)
    return new_dc_power_flows


DATOS DEL SISTEMA IEEE-14 BUS

In [42]:
# Sistema de prueba IEEE 14 bus
# linedata = [De_Bus  A_Bus  R  X  B  TR]
linedata = np.array([
    [1, 2, 0.01938, 0.05917, 0.0528, 1],
    [1, 5, 0.05403, 0.22304, 0.0492, 1],
    [2, 3, 0.04699, 0.19797, 0.0438, 1],
    [2, 4, 0.05811, 0.17632, 0.0374, 1],
    [2, 5, 0.05695, 0.17388, 0.034, 1],
    [3, 4, 0.06701, 0.17103, 0.0346, 1],
    [4, 5, 0.01335, 0.04211, 0.0128, 1],
    [4, 7, 0.0, 0.20912, 0.0, 1],
    [4, 9, 0.0, 0.55618, 0.0, 1],
    [5, 6, 0.0, 0.25202, 0.0, 1],
    [6, 11, 0.09498, 0.1989, 0.0, 1],
    [6, 12, 0.12291, 0.25581, 0.0, 1],
    [6, 13, 0.06615, 0.13027, 0.0, 1],
    [7, 8, 0.0, 0.17615, 0.0, 1],
    [7, 9, 0.0, 0.11001, 0.0, 1],
    [9, 10, 0.03181, 0.0845, 0.0, 1],
    [9, 14, 0.12711, 0.27038, 0.0, 1],
    [10, 11, 0.08205, 0.19207, 0.0, 1],
    [12, 13, 0.22092, 0.19988, 0.0, 1],
    [13, 14, 0.17093, 0.34802, 0.0, 1]
])
#Potencias en las barras, + es generada, - es consumida
#power_injected = [De_Bus  MW]
potencia_barras = np.array([
    [2, 42.4-21.7],
    [3, 23.4-94.2],
    [4, -47.8],
    [5, -7.6],
    [6, 12.2-11.2],
    [7, 0],
    [8, 17.4],
    [9, -29.5],
    [10, -9],
    [11, -3.5],
    [12, -6.1],
    [13, -13.5],
    [14, -14.9]
])
Sbase = 100  # Potencia base del sistema en MVA
Vbase = 60  # Voltaje base del sistema en kV (Lado de alta tensión)
#Contingencias por salida de generador
#ni = Bus en el que se encuentra el generador
ni = 2
Zbase = (Vbase ** 2) / Sbase
#Contingencias por salida de línea
#Salina de la línea lnm
#nm, nn = [De_Bus  A_Bus]
nm = 0
nn = 1

USO DE FUNCIONES CON DATOS DEL SISTEMA DE PRUEBA

Flujo de potencia inicial

In [43]:
B_matrix, B_matrix_result = calculate_B_matrix(linedata, Sbase, Vbase)
Theta = calculate_theta(B_matrix_result, potencia_barras)
dc_power_flows = calculate_dc_power_flows(B_matrix, linedata)
dc_power_flows_tabla = np.hstack((linedata[:, (0, 1)], dc_power_flows))
#Muestra las potencias activas de los elementos en una tabla
dc_power_flows_tabla = pd.DataFrame(dc_power_flows_tabla)
dc_power_flows_tabla.columns = ['De', 'A', 'P DC']
print(dc_power_flows_tabla)

      De     A      P DC
0    1.0   2.0  1.092783
1    1.0   5.0  0.543217
2    2.0   3.0  0.537253
3    2.0   4.0  0.437597
4    2.0   5.0  0.324932
5    3.0   4.0 -0.170747
6    4.0   5.0 -0.490571
7    4.0   7.0  0.155678
8    4.0   9.0  0.123743
9    5.0   6.0  0.301579
10   6.0  11.0  0.064699
11   6.0  12.0  0.075694
12   6.0  13.0  0.171185
13   7.0   8.0 -0.174000
14   7.0   9.0  0.329678
15   9.0  10.0  0.060301
16   9.0  14.0  0.098121
17  10.0  11.0 -0.029699
18  12.0  13.0  0.014694
19  13.0  14.0  0.050879


Flujo de potencia ante salida del generador

In [44]:
F = calculate_F(B_matrix_result)
ali = calculate_ali(linedata,B_matrix,F,ni)
new_dc_power_flows = calculate_new_dc_power_flows(linedata, dc_power_flows, potencia_barras,Sbase,ni,ali)

new_dc_power_flows_tabla = np.hstack((linedata[:, (0, 1)], new_dc_power_flows))
#Muestra las potencias activas de los elementos en una tabla
dc_power_flows_tabla = pd.DataFrame(new_dc_power_flows_tabla)
dc_power_flows_tabla.columns = ['De', 'A', 'P DC']
print(dc_power_flows_tabla)


      De     A      P DC
0    1.0   2.0  1.266254
1    1.0   5.0  0.576746
2    2.0   3.0  0.531593
3    2.0   4.0  0.425752
4    2.0   5.0  0.308909
5    3.0   4.0 -0.176407
6    4.0   5.0 -0.507135
7    4.0   7.0  0.155080
8    4.0   9.0  0.123400
9    5.0   6.0  0.302520
10   6.0  11.0  0.065266
11   6.0  12.0  0.075777
12   6.0  13.0  0.171477
13   7.0   8.0 -0.174000
14   7.0   9.0  0.329080
15   9.0  10.0  0.059734
16   9.0  14.0  0.097746
17  10.0  11.0 -0.030266
18  12.0  13.0  0.014777
19  13.0  14.0  0.051254


Flujo de potencia ante salida de linea

In [45]:
dlk = calculate_dlk(linedata, B_matrix, F)
new_dc_power_flows = new_dc_power_flows_linea(linedata, dc_power_flows, dlk, nm, nn)

new_dc_power_flows_tabla = np.hstack((linedata[:, (0, 1)], new_dc_power_flows))
#Muestra las potencias activas de los elementos en una tabla
new_dc_power_flows_tabla = pd.DataFrame(new_dc_power_flows_tabla)
new_dc_power_flows_tabla.columns = ['De', 'A', 'P DC']
print(new_dc_power_flows_tabla)


      De     A      P DC
0    1.0   2.0  3.903303
1    1.0   5.0  1.086435
2    2.0   3.0  0.445550
3    2.0   4.0  0.245683
4    2.0   5.0  0.065332
5    3.0   4.0 -0.262450
6    4.0   5.0 -0.758939
7    4.0   7.0  0.145989
8    4.0   9.0  0.118184
9    5.0   6.0  0.316827
10   6.0  11.0  0.073881
11   6.0  12.0  0.077043
12   6.0  13.0  0.175903
13   7.0   8.0 -0.174000
14   7.0   9.0  0.319989
15   9.0  10.0  0.051119
16   9.0  14.0  0.092054
17  10.0  11.0 -0.038881
18  12.0  13.0  0.016043
19  13.0  14.0  0.056946
